In [2]:
import requests
import pandas as pd
from tqdm import trange, tqdm
import numpy as np

In [3]:
# authentication 
def get_token(): 
    client_id = 'a1d86edf43894b67a642e6a5627af4ef'
    client_secret = 'm2CLw0SPFA3k8cN6OSf0XSY6P4OLlY1y'

    data = { 'grant_type': 'client_credentials' } 
    res = requests.post('https://oauth.battle.net/token', data = data, auth = (client_id, client_secret)).json()
    restoken = res['access_token']
    
    return restoken



In [4]:
def get_items(name, region, realmSlug): 
    namespace = 'profile-' + region
    locale = {'eu' : 'en_GB', 'us' : 'en_US', 'cn' : 'zh_CN'}
    
    itemsURI = 'https://' + region + '.api.blizzard.com/profile/wow/character/' + realmSlug + '/' + name + '/' + 'equipment?namespace=' + namespace + '&locale=' + locale[region] +'&access_token=' + token 
    
    #print(itemsURI)
    resG = requests.get(itemsURI).json()
    return resG

In [5]:
# need to get list of all guilds in hall of fame along with their regions 
token = get_token()

def get_guilds(token): 
    hofA_URI = 'https://us.api.blizzard.com/data/wow/leaderboard/hall-of-fame/vault-of-the-incarnates/alliance?namespace=dynamic-us&locale=en_US&access_token=' + token
    hofH_URI = 'https://us.api.blizzard.com/data/wow/leaderboard/hall-of-fame/vault-of-the-incarnates/horde?namespace=dynamic-us&locale=en_US&access_token=' + token
    
    resH = requests.get(hofH_URI).json()
    resA = requests.get(hofA_URI).json()
    
    dfH = pd.json_normalize(resH, record_path = 'entries')
    dfA = pd.json_normalize(resA, record_path = 'entries')
    
    return [dfH, dfA]
# get roster of all players in hof guilds


guilds = pd.concat(get_guilds(token))
guilds = guilds.drop(labels = ['rank', 'timestamp', 'faction.type'], axis =1) 
guilds = guilds.rename(columns={'guild.name' : 'Name', 'guild.id' : 'id' , 'guild.realm.name' : 'realm', 'guild.realm.id' : 'realm_id', 'guild.realm.slug' : 'realm_slug'})
guilds = guilds.loc[guilds['region'].isin(['eu', 'us'])].reset_index(drop = True)

In [6]:
# get players from guilds: 
# gonna get players, then for every item it needs to

def get_players(region, guildSlug, realmSlug, token): 
    locale = {'eu' : 'en_GB', 'us' : 'en_US', 'cn' : 'zh_CN'}
    namespace = 'profile-' + region
    
    rosterURI = 'https://' + region + '.api.blizzard.com/data/wow/guild/' + realmSlug + '/' + guildSlug + '/' + 'roster?namespace=' + namespace + '&locale=' + locale[region] +'&access_token=' + token 
     
    resG = requests.get(rosterURI).json()
    
    return resG 



In [7]:
locale = {'eu' : 'en_GB', 'us' : 'en_US', 'cn' : 'zh_CN', 'tw' : 'zh_TW', 'kr' : 'ko_KR'}
item_int = ['HEAD', 'SHOULDER', 'CHEST', 'LEGS'] 
classes = { 
     1: 'Warrior', 
     2: 'Paladin', 
     3: 'Hunter', 
     4: 'Rogue', 
     5: 'Priest', 
     6: 'Death Knight',
     7: 'Shaman',
     8: 'Mage',
     9: 'Warlock',
     10: 'Monk',
     11: 'Druid',
     12: 'Demon Hunter',
     13: 'Evoker'
}
players, itemslist = [], [] 

itemColumns = ['Helmet Name', 'Helment ID', 'Shoulders Name', 'Shoulders ID', 'Chest Name', 
               'Chest ID', 'Legs Name', 'Legs ID', 'characterID']

for r in tqdm(guilds.iloc):
    dat = [r[0], r[1].lower().replace(" ", "-"), r[5].lower()]
    roster = get_players(dat[0], dat[1], dat[2], token)
    dfP = pd.json_normalize(roster, record_path = 'members')
    dfP = dfP[dfP['character.level'] == 70]
    for j in range(dfP.shape[0]): 
        name = dfP['character.name'].iloc[j].lower()
        region = dat[0]
        realmSlug = dfP['character.realm.slug'].iloc[j]
        try:
            print(j, name)
            items = get_items(name, region, realmSlug)
            charID = items['character']['id']
            testDF = pd.json_normalize(items, record_path = 'equipped_items')
            reduceDF = testDF[testDF['slot.type'].isin(item_int)]
            reduceDF = reduceDF[['name', 'item.id']]
            reduceDF = np.append(reduceDF.to_numpy().flatten(), charID)
            itemRow = pd.DataFrame(reduceDF.reshape(-1, len(reduceDF)), columns = itemColumns)
            itemslist.append(itemRow)
        except: 
            continue
    meta_data = {'region': dat[0], 'guildID': r[2]}
    dfP = dfP.assign(**meta_data)
    players.append(dfP)
dfPlayers = pd.concat(players).reset_index(drop = True)
dfItems = pd.concat(itemslist).reset_index(drop = True)


dfPlayers['class'] = dfPlayers['character.playable_class.id'].map(classes)
dfPlayers = dfPlayers.drop(['character.key.href', 'character.realm.key.href', 'character.playable_class.key.href', 'character.playable_race.key.href', 'rank'], axis= 1)
dfPlayers.rename(columns={"character.name": "character_name", "character.realm.id": "realm_id", "character.id": "id"}, errors="raise")

0it [00:00, ?it/s]

0 scrype
1 scripper
2 hellrix
3 narcolíes
4 nnoggawar
5 nnogguura
6 revery
7 nnoggieh
8 nnoggiepala
9 nnoggiemonk
10 rewvez
11 ncls
12 nnoggielock
13 fragiko
14 nnoggxe
15 justwaitm
16 wõrstplâÿêr
17 qooning
18 perfectodk
19 reviledh
20 justwaith
21 rogerina
22 narcolies
23 gingidruid
24 dotadh
25 xerwo
26 gingì
27 perfectó
28 xerwelz
29 perfectohunt
30 zenwo
31 rogerush
32 meeres
33 fleksshades
34 swagstép
35 nnoggel
36 flêkspriest
37 swagfist
38 flêkstwo
39 kushington
40 xerwogrip
41 zaeliax
42 klosterbror
43 revwez
44 kushixo
45 swagshôt
46 soccrodux
47 xerwodruid
48 xerwor
49 miniw
50 minidh
51 xerwshock
52 nnoggziah
53 mêerês
54 gushforkush
55 nnoggiep
56 nnoggiedh
57 narcolie
58 xrwo
59 rogerwhite
60 rogeritsa
61 méères
62 meerês
63 meerès
64 meerés
65 scripezor
66 knüppler
67 miniww
68 hjerterdamen
69 zûgzügøøkøøk
70 meères
71 dradux
72 floorpov
73 rycn
74 rycns
75 miniaugdk
76 rogerborne
77 rogerhigh
78 rogtarogar
79 rogeroll
80 rogerfurry
81 naowhpog
82 neuermain
83 xerwotwo
8

1it [02:09, 129.44s/it]

0 dcanew
1 pinkdude
2 wommy
3 womy
4 duckie
5 smackedw
6 glaxqt
7 lilgaycane
8 bubbarog
9 pallycane
10 gatlas
11 burtleshirt
12 daggiediego
13 jeffreyxo
14 volcanohands
15 deemø
16 bubbadubstep
17 bubbledubb
18 neytirilol
19 lukezz
20 dietchrist
21 ewa
22 demoncane
23 bosstag
24 dogformx
25 bubbaduh
26 bubbadub
27 trillhunt
28 kamsdhunter
29 seeming
30 trillwar
31 seenin
32 kamshams
33 kamkin
34 kamspriest
35 inchwormer
36 dinglebop
37 popdruid
38 thdmage
39 thdemon
40 thdlocka
41 tnatty
42 thdhunt
43 thdeath
44 thdpally
45 thdshaman
46 thdmoon
47 thdrogue
48 iuke
49 thdpriest
50 lipshaman
51 daredruíd
52 maeveytwo
53 sanghelios
54 sang
55 minionlover
56 lip
57 sangheilios
58 maeveycakes
59 podra
60 ericsmage
61 genguu
62 kamslock
63 kamsmage
64 drineys
65 drineym
66 kíngflydh
67 moonfiredup
68 rukhran
69 trillpal
70 skreehelios
71 kíngfly
72 lumidelle
73 twistedr
74 twistedwl
75 lipmage
76 googoogah
77 syrethxd
78 eyelidpasta
79 goophunt
80 gooppal
81 gooplock
82 goopred
83 stuart
84 

2it [08:42, 284.68s/it]

...

0 resbear
1 trusti
2 kruse
3 potentialw
4 kaidogs
5 ilumaa
6 rimwaztok
7 twistyxd
8 jshaydh
9 netherr
10 harayeet
11 bigsalmon
12 ilumhunter
13 virginnerdx
14 flightbm
15 haralock
16 euthxyz
17 meatpack
18 gwdz
19 jshidder
20 catdom
21 chinyah
22 freshdruidd
23 acegay
24 wishcat
25 freshwar
26 moneystore
27 tofudog
28 stovielel
29 eliaas
30 vaq
31 tacoos
32 norianar
33 mucks
34 liszp
35 kaiboss
36 euthhunt
37 alterian
38 kaityn
39 gronkgodx
40 wishbun
41 jôrd
42 wishmate
43 jackwar
44 coffeeshock
45 pnpm
46 spodermin
47 freshlockk
48 haraspin
49 kozzus
50 overtninteen
51 kazansky
52 nuukd
53 potatopouch
54 littlesen
55 jayjayjaypr
56 muge
57 daebomb
58 hatchixd
59 arcadis
60 hatchonky
61 hatchoo
62 faunalotté
63 coffeê
64 fruitpouch
65 sugmabolds
66 rimhazapalla
67 picabar
68 edelman
69 juicybussy
70 coffeevoker
71 imdragondeez
72 saiira
73 noristrazza
74 longdays
75 twistyrawr
76 vacky
77 sylreth
78 applezl
79 haralol
80 smokeandfukk
81 jshay
82 freshevok
83 trustl
84 boquonkis
85 jak

178it [5:04:05, 124.31s/it]

0 hotz
1 xodeus
2 erance
3 hemer
4 meritys
5 reflx
6 fünzo
7 bambooshootr
8 spliff
9 pumfbloody
10 monkpls
11 sapphic
12 gandell
13 cian
14 unknownm
15 xodeush
16 sheepandbake
17 sakurita
18 arthuor
19 tormund
20 makederp
21 gomes
22 felwink
23 parabola
24 gangstalk
25 pytix
26 eddyravioli
27 xoddh
28 barbarajane
29 deadarth
30 buzzox
31 ravolio
32 letsgetlit
33 voidfister
34 ravà
35 clandria
36 interficio
37 pøchinki
38 tassoo
39 aldrevon
40 ratdrevon
41 listed
42 kinnersww
43 flippym
44 kinneth
45 flippyw
46 flippyr
47 flippyd
48 nakorah
49 lutherhuss
50 moama
51 spaghed
52 aitara
53 berlingr
54 vilify
55 plumsie
56 nööts
57 tricksx
58 kinners
59 arthpally
60 kinnersr
61 arthdruid
62 shwonky
63 deftrap
64 coronakun
65 biggestkritz
66 zaffe
67 shonkie
68 thebench
69 falk
70 shockydin
71 penzkì
72 quarantini
73 arthdh
74 chaediir
75 jamieb
76 faerdiir
77 liaelia
78 catdrevon
79 ænastaria
80 deffdh
81 ràava
82 flippyh
83 notamemename
84 thrahl
85 jamiebqt
86 arthmonk
87 thrayal
88 aphma

179it [5:06:20, 102.68s/it]


,character_name,id,realm_id,character.realm.slug,character.level,character.playable_class.id,character.playable_race.id,region,guildID,class
0,Scrype,97402428,1306,tarren-mill,70,8,8,eu,75163955,Mage
1,Scripper,97402881,1306,tarren-mill,70,4,5,eu,75163955,Rogue
2,Hellrix,97584505,1306,tarren-mill,70,9,2,eu,75163955,Warlock
3,Narcolíes,97585598,1306,tarren-mill,70,1,8,eu,75163955,Warrior
4,Nnoggawar,97640125,1306,tarren-mill,70,1,6,eu,75163955,Warrior
...,...,...,...,...,...,...,...,...,...,...
32977,Sharkyrogue,203097368,3725,frostmourne,70,4,4,us,66926949,Rogue
32978,Psykosys,203177782,3725,frostmourne,70,6,1,us,66926949,Death Knight
32979,Jamiebp,203201872,3725,frostmourne,70,5,1,us,66926949,Priest
32980,Aphmonk,203224491,3725,frostmourne,70,10,29,us,66926949,Monk


In [58]:
dfPlayers.to_csv('players.csv', index = False)

In [59]:
guilds.to_csv('guilds.csv', index = False)

In [6]:
# plan for getting items
# for character name, region, realm (NSpace, LOCALE, can calcualte oursevles)

# loop through each character, get their items, return ROW of HELM, SHOULDERS, PANTS, CHEST

def get_items(name, region, realmSlug): 
    namespace = 'profile-' + region
    locale = {'eu' : 'en_GB', 'us' : 'en_US', 'cn' : 'zh_CN'}
    
    itemsURI = 'https://' + region + '.api.blizzard.com/profile/wow/character/' + realmSlug + '/' + name + '/' + 'equipment?namespace=' + namespace + '&locale=' + locale[region] +'&access_token=' + token 
    
    #print(itemsURI)
    resG = requests.get(itemsURI).json()
    return resG


# EVERYTHING HAS TO BE LOWER
test =  get_items('kneem', 'us', 'sargeras')

testDF = pd.json_normalize(test, record_path = 'equipped_items')

#print(test)

print(test['character']['id'])
    



191662170


In [55]:
dfPlayers = dfPlayers.rename(columns={"character.id": "id"}, errors = "raise") 

In [7]:
dfPlayers

NameError: name 'dfPlayers' is not defined

In [120]:
reduceDF.to_numpy().flatten()

#schema name, ID, SLOT, name, ID, SLOT, name, ID, SLOT, name ID, SLOT



array(['Flaring Cowl', 193494, 'HEAD', 'Mantle of the Waking Fist',
       200365, 'SHOULDER', 'Chestwrap of the Waking Fist', 200360,
       'CHEST', 'Legguards of the Waking Fist', 200364, 'LEGS'],
      dtype=object)

In [61]:
dfPlayers

,character_name,id,realm_id,character.realm.slug,character.level,character.playable_class.id,character.playable_race.id,region,guildID
0,Reviledk,96746595,1306,tarren-mill,60,6,10,eu,75163955
1,Thug,97184878,1306,tarren-mill,60,2,10,eu,75163955
2,Toscar,97397454,1306,tarren-mill,60,7,8,eu,75163955
3,Scrype,97402428,1306,tarren-mill,70,8,8,eu,75163955
4,Scripper,97402881,1306,tarren-mill,70,4,5,eu,75163955
...,...,...,...,...,...,...,...,...,...
85564,Sharkyrogue,203097368,3725,frostmourne,70,4,4,us,66926949
85565,Psykosys,203177782,3725,frostmourne,70,6,1,us,66926949
85566,Jamiebp,203201872,3725,frostmourne,70,5,1,us,66926949
85567,Aphmonk,203224491,3725,frostmourne,70,10,29,us,66926949


In [6]:
 classes = { 
     1: 'Warrior', 
     2: 'Paladin', 
     3: 'Hunter', 
     4: 'Rogue', 
     5: 'Priest', 
     6: 'Death Knight',
     7: 'Shaman',
     8: 'Mage',
     9: 'Warlock',
     10: 'Monk',
     11: 'Druid',
     12: 'Demon Hunter',
     13: 'Evoker'
 }
    

dfPlayers['class'] = dfPlayers['character.playable_class.id'].map(classes)

NameError: name 'dfPlayers' is not defined

In [1]:
reduceDF

NameError: name 'reduceDF' is not defined

In [10]:
dfItems.to_csv('items.csv', index = False)

In [11]:
dfItems

,Helmet Name,Helment ID,Shoulders Name,Shoulders ID,Chest Name,Chest ID,Legs Name,Legs ID,characterID
0,Crystal Scholar's Cowl,200318,Amice of the Blue,193526,Crystal Scholar's Tunic,200315,Crystal Scholar's Britches,200319,97402428
1,Ottuk Hide Helm,191993,Ottuk Hide Epaulets,192014,Invader's Firestorm Chestguard,193764,Ottuk Hide Breeches,191994,97402881
2,Scalesworn Cultist's Scorn,200336,Vibrant Wildercloth Shoulderspikes,193508,Scalesworn Cultist's Frock,200333,Scalesworn Cultist's Culottes,200337,97584505
3,Qalashi War-Helm,200757,Peaks of the Walking Mountain,200428,Husk of the Walking Mountain,200423,Poleyns of the Walking Mountain,200427,97585598
4,Casque of the Walking Mountain,200426,Peaks of the Walking Mountain,200428,Explorer's Plate Chestguard,189538,Poleyns of the Walking Mountain,200427,97640125
...,...,...,...,...,...,...,...,...,...
32904,Ottuk Hide Helm,191993,Vault Delver's Epaulets,200374,Vault Delver's Brigandine,200369,Vault Delver's Pantaloons,200373,203097368
32905,Goldsteel Greathelm,109987,Jaws of the Haunted Frostbrood,200410,Breastplate of the Haunted Frostbrood,200405,Greaves of the Haunted Frostbrood,200409,203177782
32906,Draconic Hierophant's Archcowl,200327,Draconic Hierophant's Wisdom,200329,Draconic Hierophant's Vestment,200324,Draconic Hierophant's Britches,200328,203201872
32907,Pioneer's Practiced Cowl,201941,Crimson Combatant's Resilient Shoulderpads,193433,Crimson Combatant's Resilient Chestpiece,193431,Pioneer's Practiced Leggings,201944,203224491


In [14]:
dfPlayersF = pd.concat([dfPlayers, dfItems], axis=1, join="inner")


In [15]:
dfPlayersF

,character.name,character.id,character.realm.id,character.realm.slug,character.level,character.playable_class.id,character.playable_race.id,region,guildID,class,Helmet Name,Helment ID,Shoulders Name,Shoulders ID,Chest Name,Chest ID,Legs Name,Legs ID,characterID
0,Scrype,97402428,1306,tarren-mill,70,8,8,eu,75163955,Mage,Crystal Scholar's Cowl,200318,Amice of the Blue,193526,Crystal Scholar's Tunic,200315,Crystal Scholar's Britches,200319,97402428
1,Scripper,97402881,1306,tarren-mill,70,4,5,eu,75163955,Rogue,Ottuk Hide Helm,191993,Ottuk Hide Epaulets,192014,Invader's Firestorm Chestguard,193764,Ottuk Hide Breeches,191994,97402881
2,Hellrix,97584505,1306,tarren-mill,70,9,2,eu,75163955,Warlock,Scalesworn Cultist's Scorn,200336,Vibrant Wildercloth Shoulderspikes,193508,Scalesworn Cultist's Frock,200333,Scalesworn Cultist's Culottes,200337,97584505
3,Narcolíes,97585598,1306,tarren-mill,70,1,8,eu,75163955,Warrior,Qalashi War-Helm,200757,Peaks of the Walking Mountain,200428,Husk of the Walking Mountain,200423,Poleyns of the Walking Mountain,200427,97585598
4,Nnoggawar,97640125,1306,tarren-mill,70,1,6,eu,75163955,Warrior,Casque of the Walking Mountain,200426,Peaks of the Walking Mountain,200428,Explorer's Plate Chestguard,189538,Poleyns of the Walking Mountain,200427,97640125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32904,Catdrevon,184119116,3725,frostmourne,70,11,4,us,66926949,Druid,Ottuk Hide Helm,191993,Vault Delver's Epaulets,200374,Vault Delver's Brigandine,200369,Vault Delver's Pantaloons,200373,203097368
32905,Ænastaria,184181074,3725,frostmourne,70,11,4,us,66926949,Druid,Goldsteel Greathelm,109987,Jaws of the Haunted Frostbrood,200410,Breastplate of the Haunted Frostbrood,200405,Greaves of the Haunted Frostbrood,200409,203177782
32906,Deffdh,184240946,3725,frostmourne,70,12,4,us,66926949,Demon Hunter,Draconic Hierophant's Archcowl,200327,Draconic Hierophant's Wisdom,200329,Draconic Hierophant's Vestment,200324,Draconic Hierophant's Britches,200328,203201872
32907,Ràava,184334299,3725,frostmourne,70,5,29,us,66926949,Priest,Pioneer's Practiced Cowl,201941,Crimson Combatant's Resilient Shoulderpads,193433,Crimson Combatant's Resilient Chestpiece,193431,Pioneer's Practiced Leggings,201944,203224491


In [17]:
dfPlayers

,character.name,character.id,character.realm.id,character.realm.slug,character.level,character.playable_class.id,character.playable_race.id,region,guildID,class
0,Scrype,97402428,1306,tarren-mill,70,8,8,eu,75163955,Mage
1,Scripper,97402881,1306,tarren-mill,70,4,5,eu,75163955,Rogue
2,Hellrix,97584505,1306,tarren-mill,70,9,2,eu,75163955,Warlock
3,Narcolíes,97585598,1306,tarren-mill,70,1,8,eu,75163955,Warrior
4,Nnoggawar,97640125,1306,tarren-mill,70,1,6,eu,75163955,Warrior
...,...,...,...,...,...,...,...,...,...,...
32977,Sharkyrogue,203097368,3725,frostmourne,70,4,4,us,66926949,Rogue
32978,Psykosys,203177782,3725,frostmourne,70,6,1,us,66926949,Death Knight
32979,Jamiebp,203201872,3725,frostmourne,70,5,1,us,66926949,Priest
32980,Aphmonk,203224491,3725,frostmourne,70,10,29,us,66926949,Monk


In [21]:
dfPlayersF[['character.id', 'characterID']]

,character.id,characterID
0,97402428,97402428
1,97402881,97402881
2,97584505,97584505
3,97585598,97585598
4,97640125,97640125
...,...,...
32904,184119116,203097368
32905,184181074,203177782
32906,184240946,203201872
32907,184334299,203224491


In [25]:
dfPlayersF


,character.name,character.id,character.realm.id,character.realm.slug,character.level,character.playable_class.id,character.playable_race.id,region,guildID,class,Helmet Name,Helment ID,Shoulders Name,Shoulders ID,Chest Name,Chest ID,Legs Name,Legs ID,characterID
0,Scrype,97402428,1306,tarren-mill,70,8,8,eu,75163955,Mage,Crystal Scholar's Cowl,200318,Amice of the Blue,193526,Crystal Scholar's Tunic,200315,Crystal Scholar's Britches,200319,97402428
1,Scripper,97402881,1306,tarren-mill,70,4,5,eu,75163955,Rogue,Ottuk Hide Helm,191993,Ottuk Hide Epaulets,192014,Invader's Firestorm Chestguard,193764,Ottuk Hide Breeches,191994,97402881
2,Hellrix,97584505,1306,tarren-mill,70,9,2,eu,75163955,Warlock,Scalesworn Cultist's Scorn,200336,Vibrant Wildercloth Shoulderspikes,193508,Scalesworn Cultist's Frock,200333,Scalesworn Cultist's Culottes,200337,97584505
3,Narcolíes,97585598,1306,tarren-mill,70,1,8,eu,75163955,Warrior,Qalashi War-Helm,200757,Peaks of the Walking Mountain,200428,Husk of the Walking Mountain,200423,Poleyns of the Walking Mountain,200427,97585598
4,Nnoggawar,97640125,1306,tarren-mill,70,1,6,eu,75163955,Warrior,Casque of the Walking Mountain,200426,Peaks of the Walking Mountain,200428,Explorer's Plate Chestguard,189538,Poleyns of the Walking Mountain,200427,97640125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32904,Catdrevon,184119116,3725,frostmourne,70,11,4,us,66926949,Druid,Ottuk Hide Helm,191993,Vault Delver's Epaulets,200374,Vault Delver's Brigandine,200369,Vault Delver's Pantaloons,200373,203097368
32905,Ænastaria,184181074,3725,frostmourne,70,11,4,us,66926949,Druid,Goldsteel Greathelm,109987,Jaws of the Haunted Frostbrood,200410,Breastplate of the Haunted Frostbrood,200405,Greaves of the Haunted Frostbrood,200409,203177782
32906,Deffdh,184240946,3725,frostmourne,70,12,4,us,66926949,Demon Hunter,Draconic Hierophant's Archcowl,200327,Draconic Hierophant's Wisdom,200329,Draconic Hierophant's Vestment,200324,Draconic Hierophant's Britches,200328,203201872
32907,Ràava,184334299,3725,frostmourne,70,5,29,us,66926949,Priest,Pioneer's Practiced Cowl,201941,Crimson Combatant's Resilient Shoulderpads,193433,Crimson Combatant's Resilient Chestpiece,193431,Pioneer's Practiced Leggings,201944,203224491


In [27]:
dfPlayers[dfPlayers['character.id'] == 203262635]

,character.name,character.id,character.realm.id,character.realm.slug,character.level,character.playable_class.id,character.playable_race.id,region,guildID,class
32981,Crobat,203262635,3725,frostmourne,70,13,52,us,66926949,Evoker


In [28]:
dfPlayers2 = dfPlayers.copy()

In [35]:
mergedPlayers = pd.merge(dfPlayers2, dfItems, left_on = 'character.id', right_on = 'characterID')

In [43]:
# renaming columns n stuff

mergedPlayers.to_csv('players2.csv', index = False)






In [56]:
newColumns={"character.name": "character_name", 
         "character.id": "id", 
         "character.realm.id": "realm_id",
         "character.realm.slug": "character_realm_slug", 
         "character.level": "character_level", 
         "Helmet Name": "helmet_name", 
         "Helment ID": "helmet_id", 
         "Shoulders Name": "shoulders_name", 
         "Shoulders ID": "shoulders_id", 
         "Chest Name": "chest_name", 
         "Legs Name": "legs_name", 
         "Legs ID": "legs_id", 
         "Chest ID": "chest_id"}

mergedPlayers = mergedPlayers.rename(columns=newColumns, errors="raise")

KeyError: "['character.name', 'character.id', 'character.realm.id', 'character.realm.slug', 'character.level', 'Helmet Name', 'Helment ID', 'Shoulders Name', 'Shoulders ID', 'Chest Name', 'Legs Name', 'Legs ID', 'Chest ID'] not found in axis"

In [64]:
mergedPlayers.to_csv('players2.csv', index = False)

In [62]:
mergedPlayers = mergedPlayers.rename(columns = {"class": "player_class"})

In [63]:
mergedPlayers

,character_name,id,realm_id,character_realm_slug,character_level,character.playable_class.id,character.playable_race.id,region,guildID,player_class,helmet_name,helmet_id,shoulders_name,shoulders_id,chest_name,chest_id,legs_name,legs_id,characterID
0,Scrype,97402428,1306,tarren-mill,70,8,8,eu,75163955,Mage,Crystal Scholar's Cowl,200318,Amice of the Blue,193526,Crystal Scholar's Tunic,200315,Crystal Scholar's Britches,200319,97402428
1,Scripper,97402881,1306,tarren-mill,70,4,5,eu,75163955,Rogue,Ottuk Hide Helm,191993,Ottuk Hide Epaulets,192014,Invader's Firestorm Chestguard,193764,Ottuk Hide Breeches,191994,97402881
2,Hellrix,97584505,1306,tarren-mill,70,9,2,eu,75163955,Warlock,Scalesworn Cultist's Scorn,200336,Vibrant Wildercloth Shoulderspikes,193508,Scalesworn Cultist's Frock,200333,Scalesworn Cultist's Culottes,200337,97584505
3,Narcolíes,97585598,1306,tarren-mill,70,1,8,eu,75163955,Warrior,Qalashi War-Helm,200757,Peaks of the Walking Mountain,200428,Husk of the Walking Mountain,200423,Poleyns of the Walking Mountain,200427,97585598
4,Nnoggawar,97640125,1306,tarren-mill,70,1,6,eu,75163955,Warrior,Casque of the Walking Mountain,200426,Peaks of the Walking Mountain,200428,Explorer's Plate Chestguard,189538,Poleyns of the Walking Mountain,200427,97640125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32918,Sharkyrogue,203097368,3725,frostmourne,70,4,4,us,66926949,Rogue,Ottuk Hide Helm,191993,Vault Delver's Epaulets,200374,Vault Delver's Brigandine,200369,Vault Delver's Pantaloons,200373,203097368
32919,Psykosys,203177782,3725,frostmourne,70,6,1,us,66926949,Death Knight,Goldsteel Greathelm,109987,Jaws of the Haunted Frostbrood,200410,Breastplate of the Haunted Frostbrood,200405,Greaves of the Haunted Frostbrood,200409,203177782
32920,Jamiebp,203201872,3725,frostmourne,70,5,1,us,66926949,Priest,Draconic Hierophant's Archcowl,200327,Draconic Hierophant's Wisdom,200329,Draconic Hierophant's Vestment,200324,Draconic Hierophant's Britches,200328,203201872
32921,Aphmonk,203224491,3725,frostmourne,70,10,29,us,66926949,Monk,Pioneer's Practiced Cowl,201941,Crimson Combatant's Resilient Shoulderpads,193433,Crimson Combatant's Resilient Chestpiece,193431,Pioneer's Practiced Leggings,201944,203224491
